<a href="https://colab.research.google.com/github/ColmTalbot/gmm_sensitivity_estimation/blob/main/gmm_sensitivity_estimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Flexible and accurate evaluation of gravitational-wave Malmquist bias with machine learning

This notebook is a companion to [Talbot and Thrane](https://arxiv.org/abs/2012.01317) that produced all results in that work.

Note that the exact result will not in general be reproducible due to fluctuations in the random number generation.

This notebook is as self contained as possible, however for the last section (performing population inference) a large amount of input data is required.
Rather than download many GBs of data every time, I mount a local compressed version of the data. Instructions for how to obtain the full data set can be found at https://docs.ligo.org/RatesAndPopulations/gwpopulation_pipe/real-data.html although care should be taken in making sure all the correct events are identified.

## Setup

These first few cells will install all software needed to run this in [google colaboratory](colab.research.google.com).

If you run with colab, I recommend using a GPU runtime.

Some of the steps may not be necessary for some users or as updates get rolled out with colab. I make no guarantee that this will be the best way to use this going forward.

In [ ]:
!pip install corner bilby gwpopulation astropy nestle gwpopulation_pipe

In [ ]:
!pip uninstall matplotlib -y
!pip install update matplotlib

In [ ]:
# !wget -nc https://zenodo.org/record/5636816/files/o1%2Bo2%2Bo3_bbhpop_real%2Bsemianalytic-LIGO-T2100377-v2.hdf5
!wget -nc https://zenodo.org/record/5546676/files/endo3_bbhpop-LIGO-T2100113-v12.hdf5

In [ ]:
!sudo apt-get install texlive-latex-recommended
!sudo apt-get install dvipng texlive-latex-extra texlive-fonts-recommended
!wget http://mirrors.ctan.org/macros/latex/contrib/type1cm.zip
!unzip type1cm.zip -d /tmp/type1cm
!cd /tmp/type1cm/type1cm/ && sudo latex type1cm.ins
!sudo mkdir /usr/share/texmf/tex/latex/type1cm
!sudo cp /tmp/type1cm/type1cm/type1cm.sty /usr/share/texmf/tex/latex/type1cm
!sudo texhash
!apt install cm-super

## Preliminaries

The imports we'll use are here at the top for clarity.

Also, various matplotlib settings to make nice plots.

In [ ]:
%pylab inline

import os
from copy import deepcopy
import h5py

import bilby
from bilby.hyper.model import Model
from bilby.core.prior import joint, PriorDict, Uniform
import corner
import gwpopulation
import pandas as pd
from bilby.hyper.model import Model
from scipy.integrate import cumulative_trapezoid
from scipy.interpolate import interp1d
from scipy.special import betaln as scbetaln
from scipy.stats import gaussian_kde, ks_2samp, norm, beta, truncnorm
from sklearn.mixture import GaussianMixture
from tqdm.auto import tqdm, trange
from gwpopulation.cupy_utils import to_numpy
from gwpopulation.models.mass import two_component_primary_mass_ratio
from gwpopulation.models.spin import iid_spin
from gwpopulation.models.redshift import PowerLawRedshift
from gwpopulation.vt import ResamplingVT
from gwpopulation_pipe.utils import prior_conversion
from gwpopulation_pipe.vt_helper import load_injection_data

try:
    import cupy as xp
    from cupyx.scipy.special import erf, erfinv
except ImportError:
    xp = np
    from scipy.special import erf, erfinv

In [ ]:
mpl.rcParams["font.family"] = "serif"
mpl.rcParams["font.serif"] = "Computer Modern Roman"
mpl.rcParams["font.size"] = 20
mpl.rcParams["text.usetex"] = True
mpl.rcParams["grid.alpha"] = 0
mpl.rcParams['text.latex.preamble'] = ""

### Load in the injection data

Here we just use the found injections from the O3 injection campaign.

The defails can be found on [zenodo](https://zenodo.org/record/5546676#.Yc80s_HP23I).

In [ ]:
injections = load_injection_data("endo3_bbhpop-LIGO-T2100113-v12.hdf5")
fraction_not_hopeless = len(injections["mass_1"]) / injections.pop("total_generated")
analysis_time = injections.pop("analysis_time")
for key in injections:
    if isinstance(injections[key], xp.ndarray):
        injections[key] = to_numpy(injections[key])
all_found_injections = pd.DataFrame(injections)
all_found_injections.describe()

## "Empirical" scaling preliminaries

For our favoured scaling method we use a reflecting KDE of the found injections to construct an empirical CDF.

This cells computes those distributions and verifies that the interpolated PDF agrees with the original samples and new samples we generate.

In [ ]:
class ReflectingKDE(gaussian_kde):

    def __init__(self, dataset, bw_method=None, weights=None, minimum=-np.inf, maximum=np.inf):
        super(ReflectingKDE, self).__init__(
            dataset=dataset, bw_method=bw_method, weights=weights
        )
        self.minimum = minimum
        self.maximum = maximum
        self.range = self.maximum - self.minimum

    def __call__(self, points):
        prob = self.evaluate(points)
        fraction = (points - self.minimum) / self.range
        if self.minimum > -np.inf:
            prob += self.evaluate(self.minimum - fraction * self.range)
            prob *= (points >= self.minimum)
        if self.maximum < np.inf:
            prob += self.evaluate(self.maximum + (1 - fraction) * self.range)
            prob *= (points <= self.maximum)
        return prob


_data = deepcopy(all_found_injections)
ranges = dict(
    mass_1=(2, 100),
    mass_ratio=(0, 1),
    a_1=(0, 1),
    a_2=(0, 1),
    cos_tilt_1=(-1, 1),
    cos_tilt_2=(-1, 1),
)

lnpdf_ = dict()
cdf_ = dict()
pdf_ = dict()
ppf_ = dict()

for key in ranges:
    min_, max_ = ranges[key]
    domain = np.linspace(min_, max_, 10000)
    if key == "mass_1":
        kde = ReflectingKDE(to_numpy(_data[key]), minimum=min_, maximum=max_, weights=to_numpy(_data[key] ** 0.5))
    else:
        kde = ReflectingKDE(to_numpy(_data[key]), minimum=min_, maximum=max_)

    epdf = kde(domain)
    if key == "mass_1":
        epdf *= domain ** -0.5
        epdf /= np.trapz(epdf, domain)

    ecdf = cumulative_trapezoid(epdf, domain, initial=0)

    pdf_[key] = interp1d(domain, epdf)
    lnpdf_[key] = interp1d(domain, np.log(epdf))
    cdf_[key] = interp1d(domain, ecdf)
    ppf_[key] = interp1d(ecdf, domain)

    plt.hist(_data[key], bins=100, density=True, histtype="step")
    plt.hist(ppf_[key](np.random.uniform(0, 1, 80000)), bins=100, density=True, histtype="step")
    plt.plot(domain, epdf)
    plt.yscale("log")
    plt.show()
    plt.close()

### Injected distributions - mass

We need to be able to simulate new events from the injected distribution.
This involves defining some functions to do this.

The mass distribution of the injections is

$$
p(m_1, m_2 | \Lambda_0) = p(m_1 | \Lambda_0) p(m_2 | m_1, \Lambda_0) \\
p(m_1 | \Lambda) = \frac{1}{(1 - \alpha)} \frac{m_1^{-\alpha}}{(m_{\max}^{1 - \alpha} - m_{\min}^{1 - \alpha})} \\
p(m_2 | m_1, \Lambda) = \frac{1}{(1 + \beta)} \frac{m_2^{\beta}}{(m_{1}^{1 + \beta} - m_{\min}^{1 + \beta})}.
$$

Later we will reparameterise in terms of the primary mass and the mass ratio.
In that case the usual change of variables rule shows us
$$
p(m_1, q | \Lambda_0) = p(m_1, m_2 | \Lambda_0) \left|\frac{d m_2}{d q}\right| = m_1 p(m_1, m_2 | \Lambda_0).
$$

The specific hyperparameters used for the injections are
$$
\Lambda_0 = \{ \alpha_{0}=-2.35, \beta_{0}=2, m_{\min, 0}=2, m_{\max, 0}=100 \}.
$$

All of these distributions are simply power laws.
We need a few other features of this distribution namely the cumulative distribution function (CDF) and the percentile point function (PPF, the inverse of the CDF).
For a generic parameter $x$ power law distributed in $[a, b]$ with index $k$ we have
$$
p(x | a, b, k) = \frac{1}{(1 + k)} \frac{x^{k}}{(b^{1 + k} - a^{1 + k})} \\
CDF(x | a, b, k) = \int_{a}^{x} dx' p(x' | a, b, k) = \frac{(x^{1 + k} - a^{1 + k})}{(b^{1 + k} - a^{1 + k})} \\
PPF(u | a, b, k) = \left(a^{1 + k} + \left(b^{1 + k} - a^{1 + k}\right) u \right)^{1 / (1 + k)} = a\left(1 + \left(\left(\frac{b}{a}\right)^{1 + k} - 1\right) u \right)^{1 / (1 + k)}
$$ 

In [ ]:
def powerlaw_cdf(value, alpha, minimum, maximum):
    normalization = np.exp(powerlaw_ln_normalization(alpha, minimum, maximum))
    return (value ** (1 + alpha) - minimum ** (1 + alpha)) * normalization / (1 + alpha)


def powerlaw_ln_pdf(value, alpha, minimum, maximum):
    return alpha * np.log(value) + powerlaw_ln_normalization(alpha, minimum, maximum)


def powerlaw_ln_normalization(alpha, minimum, maximum):
    return np.log((1 + alpha) / (maximum ** (1 + alpha) - minimum ** (1 + alpha)))


def powerlaw_ppf(value, alpha, minimum, maximum):
    """
    Percentile point function for a power law distribution.
    """
    scale = (maximum / minimum) ** (alpha + 1)
    return minimum * (1 + (scale - 1) * value) ** (1 / (alpha + 1))


def powerlaw_sample(alpha, minimum, maximum, size=None, rng=xp.random):
    x_rand = rng.uniform(0, 1, size=size)
    parameters = dict(value=x_rand, alpha=alpha, minimum=minimum, maximum=maximum)
    return powerlaw_ppf(**parameters)

### Injected distributions - spin

The injections are created with aligned spin components (\chi) uniformly distribted in $[-1, 1]$.
This is a special case of a power law with $k=0, a=-1, b=1$.
We can simply write the PDF, CDF, and PPF in this case.
$$
p(\chi | \Lambda_0) = \frac{1}{2} \\
CDF(\chi | \Lambda_0) = \frac{\chi + 1}{2} \\
PPF(u | \Lambda_0) = 2 u - 1 \\
$$

### Perform the initial mapping

The distributions above are not conducive to fitting a Gaussian Mixture Model (GMM) to the data.
This is primarily because they have sharp edges and finite domains.
In order to mitigate this, we map the data onto an infinite domain in a two stage process
$$
x' = \Phi^{-1}(U(x)).
$$

The first stage $G$ maps the physical domain to the unit interval $[0, 1]$.
There are many possible ways to perform this mapping.
For demonstration we consider four methods:
- the first and simplest method performs a linear mapping over the domain $$U(x) = \frac{x - x_{\min}}{x_{\max} - x_{\min}}.$$
- the next method uses the CDF of the injected distribution $$U(x) = \int_{x_\min}^{x} dx' \, p(x' | \Lambda_{0}).$$
- the third method is an analytic approximation to the CDF of the observed distribution $$U(x) = \int_{x_\min}^{x} dx' \, p(x' | \Lambda_{0}) \hat{p}_{\rm det}(x') .$$
- finally, we consider another case where we approximate the one-dimensional observed distributions using a kernel density estimate and construct an empirical CDF.

For $\hat{p}_{\rm det}$ we choose the following
$$
\hat{p}_{\rm det}(m_1) = m_1^{2.35} \\
\hat{p}_{\rm det}(q) = q^2
$$
The mass scaling follows the leading order dependence of signal amplitude with mass and has been discussed many places in the literature.
The mass ratio scaling is somewhat ad hoc based on numerical tests of what makes the scaled samples look closest to a uniform disribution on $[0, 1]$.

The second stage maps from the unit interval onto the inifite domain $[-\infty, \infty]$ using the probit function (the inverse of the CDF of the unit normal distribution.)
As long as the underlying distribution doesn't rail exponentially against the edge of the original domain, after this transformation the data will go to zero smoothly at positive and negative infinity.

In [ ]:
def naive_scale(values, minimum, maximum):
    return (values - minimum) / (maximum - minimum)


def naive_unscale(values, minimum, maximum):
    return values * (maximum - minimum) + minimum


def call_interp(point, interpolant):
    if not hasattr(interpolant, "xcp"):
        interpolant.xcp = xp.asarray(interpolant.x)
    if not hasattr(interpolant, "ycp"):
        interpolant.ycp = xp.asarray(interpolant.y)
    delta_x = interpolant.xcp[1] - interpolant.xcp[0]
    idx = (point - interpolant.xcp[0]) // delta_x
    idx = idx.astype(int)
    fraction = (point - interpolant.xcp[0]) % delta_x
    return (
        (1 - fraction) * interpolant.ycp[idx]
        + fraction * interpolant.ycp[idx + 1]
    )



def scale_data(data, parameters, scale_method="cdf", module=xp):
    data = deepcopy(data)

    if "minimum_mass" in parameters:
        minimum_mass = parameters["minimum_mass"]
    else:
        minimum_mass = parameters["mmin"]
    if "maximum_mass" in parameters:
        maximum_mass = parameters["maximum_mass"]
    else:
        maximum_mass = parameters["mmax"]
    minimum_spin = parameters["minimum_spin"]
    maximum_spin = parameters["maximum_spin"]

    if scale_method == "cdf":

        data["a_1"] = naive_scale(data["a_1"], 0, maximum_spin)
        data["a_2"] = naive_scale(data["a_2"], 0, maximum_spin)
        data["cos_tilt_1"] = naive_scale(data["cos_tilt_1"], minimum_spin, maximum_spin)
        data["cos_tilt_2"] = naive_scale(data["cos_tilt_2"], minimum_spin, maximum_spin)

        data["mass_ratio"] = powerlaw_cdf(
            data["mass_ratio"],
            alpha=parameters["beta"],
            minimum=minimum_mass / data["mass_1"],
            maximum=1,
        )

        data["mass_1"] = powerlaw_cdf(
            data["mass_1"],
            minimum=minimum_mass,
            maximum=maximum_mass,
            alpha=parameters["alpha"] + 0
        )

    elif scale_method == "naive":

        data["a_1"] = naive_scale(data["a_1"], 0, maximum_spin)
        data["a_2"] = naive_scale(data["a_2"], 0, maximum_spin)
        data["cos_tilt_1"] = naive_scale(data["cos_tilt_1"], minimum_spin, maximum_spin)
        data["cos_tilt_2"] = naive_scale(data["cos_tilt_2"], minimum_spin, maximum_spin)

        data["mass_ratio"] = naive_scale(data["mass_ratio"], minimum_mass / data["mass_1"], 1)
        data["mass_1"] = naive_scale(data["mass_1"], minimum_mass, maximum_mass)


    elif scale_method == "empirical":
        for key in data:
            if module == np:
                data[key] = cdf_[key](data[key])
            else:
                data[key] = call_interp(xp.asarray(data[key]), cdf_[key])
            
    elif scale_method == "approximate":
        data["a_1"] = naive_scale(data["a_1"], 0, maximum_spin)
        data["a_2"] = naive_scale(data["a_2"], 0, maximum_spin)
        data["cos_tilt_1"] = naive_scale(data["cos_tilt_1"], minimum_spin, maximum_spin)
        data["cos_tilt_2"] = naive_scale(data["cos_tilt_2"], minimum_spin, maximum_spin)

        data["mass_ratio"] = powerlaw_cdf(
            data["mass_ratio"],
            alpha=parameters["beta"] + 2,
            minimum=minimum_mass / data["mass_1"],
            maximum=1,
        )

        data["mass_1"] = powerlaw_cdf(
            data["mass_1"],
            minimum=minimum_mass,
            maximum=maximum_mass,
            alpha=parameters["alpha"] + 2.35
        )

    else:
        raise ValueError(f"Invalid input {scaled_method}")

    return data


def unscale_data(data, parameters, scale_method="cdf"):
    data = deepcopy(data)

    if "minimum_mass" in parameters:
        minimum_mass = parameters["minimum_mass"]
    else:
        minimum_mass = parameters["mmin"]
    if "maximum_mass" in parameters:
        maximum_mass = parameters["maximum_mass"]
    else:
        maximum_mass = parameters["mmax"]

    if scale_method == "cdf":

        data["a_1"] = naive_unscale(data["a_1"], 0, maximum_spin)
        data["a_2"] = naive_unscale(data["a_2"], 0, maximum_spin)
        data["cos_tilt_1"] = naive_unscale(data["cos_tilt_1"], minimum_spin, maximum_spin)
        data["cos_tilt_2"] = naive_unscale(data["cos_tilt_2"], minimum_spin, maximum_spin)

        data["mass_1"] = powerlaw_ppf(
            data["mass_1"],
            minimum=minimum_mass,
            maximum=maximum_mass,
            alpha=parameters["alpha"] + 0
        )

        data["mass_ratio"] = powerlaw_ppf(
            data["mass_ratio"],
            alpha=parameters["beta"],
            minimum=minimum_mass / data["mass_1"],
            maximum=1,
        )

    elif scale_method == "naive":

        data["a_1"] = naive_unscale(data["a_1"], 0, maximum_spin)
        data["a_2"] = naive_unscale(data["a_2"], 0, maximum_spin)
        data["cos_tilt_1"] = naive_unscale(data["cos_tilt_1"], minimum_spin, maximum_spin)
        data["cos_tilt_2"] = naive_unscale(data["cos_tilt_2"], minimum_spin, maximum_spin)

        data["mass_1"] = naive_unscale(data["mass_1"], minimum_mass, maximum_mass)
        data["mass_ratio"] = naive_unscale(data["mass_ratio"], minimum_mass / data["mass_1"], 1)

    elif scale_method == "empirical":

        for key in data:
            data[key] = xp.asarray(ppf_[key](to_numpy(data[key])))

    elif scale_method == "approximate":
        data["a_1"] = naive_unscale(data["a_1"], 0, maximum_spin)
        data["a_2"] = naive_unscale(data["a_2"], 0, maximum_spin)
        data["cos_tilt_1"] = naive_unscale(data["cos_tilt_1"], minimum_spin, maximum_spin)
        data["cos_tilt_2"] = naive_unscale(data["cos_tilt_2"], minimum_spin, maximum_spin)

        data["mass_1"] = powerlaw_ppf(
            data["mass_1"],
            minimum=minimum_mass,
            maximum=maximum_mass,
            alpha=parameters["alpha"] + 2.35
        )

        data["mass_ratio"] = powerlaw_ppf(
            data["mass_ratio"],
            alpha=parameters["beta"] + 2,
            minimum=minimum_mass / data["mass_1"],
            maximum=1,
        )

    else:
        raise ValueError(f"Invalid input {scaled_method}")

    return data

### Plot the original and scaled data.

This is Figure 1 of the paper.

In [ ]:
from corner import hist2d as chist2d

_labels = dict(
    mass_1="$m_{1}$",
    mass_ratio="$q$",
    a_1="$a_1$",
    a_2="$a_2$",
    cos_tilt_1="$\\cos\\theta_1$",
    cos_tilt_2="$\\cos\\theta_2$",
    chi_1="$\\chi_1$",
    chi_2="$\\chi_2$",
)

injection_parameters = dict(
    alpha=-2.35,
    beta=1,
    minimum_mass=2,
    maximum_mass=100,
    minimum_spin=-1,
    maximum_spin=1,
)

mpl.rcParams["font.size"] = 30

fit_keys = ["mass_1", "mass_ratio", "a_1", "a_2", "cos_tilt_1", "cos_tilt_2"]

fig, axs = plt.subplots(nrows=5, ncols=len(fit_keys) - 1, figsize=(5 * len(fit_keys) - 1, 25))

for ii in range(len(fit_keys) - 1):
    key_1 = fit_keys[ii]
    key_2 = fit_keys[ii + 1]
    chist2d(
        all_found_injections[key_1].values, all_found_injections[key_2].values,
        bins=50, smooth=True, color="C0",
        plot_density=False, plot_datapoints=False, fill_contours=True,
        ax=axs[0][ii],
    )
    axs[0][ii].set_xlabel(_labels[key_1])
    axs[0][ii].set_ylabel(_labels[key_2])

scaled_data = dict()
scalings = dict(naive="Naive", cdf="CDF", approximate="Approximate", empirical="Empirical")

for jj, scale_method in enumerate(["naive", "cdf", "approximate", "empirical"]):

    data = scale_data(
        all_found_injections.copy()[fit_keys],
        parameters=injection_parameters,
        scale_method=scale_method,
        module=np,
    )
    scaled_data[scale_method] = data
    original_samples = norm.ppf(np.array(data[fit_keys].values))

    for ii in range(len(fit_keys) - 1):
        key_1 = fit_keys[ii]
        key_2 = fit_keys[ii + 1]
        chist2d(
            original_samples[:, ii], original_samples[:, ii + 1], 
            bins=50, smooth=True, color=f"C{jj + 1}",
            plot_density=False, plot_datapoints=False, fill_contours=True,
            ax=axs[jj + 1][ii],
        )
        axs[jj + 1][ii].set_xlabel(f"{scalings[scale_method]} {_labels[key_1]}")
        axs[jj + 1][ii].set_ylabel(f"{scalings[scale_method]} {_labels[key_2]}")
        axs[jj + 1][ii].set_xlim(-3, 3)
        axs[jj + 1][ii].set_ylim(-3, 3)

axs[0][0].set_xlabel("$m_{1} [M_{\\odot}]$")
axs[1][1].set_xlim(-1, 3)
axs[2][0].set_xlim(0, 4)
axs[1][0].set_ylim(-1, 3)
plt.tight_layout()
plt.savefig("scaling-comparison.pdf")
plt.show()
plt.close()
mpl.rcParams["font.size"] = 20

We'll also look at the distribution of the found injections after the initial unit mapping. This tells us how well each method does at approximating the observed one-dimensional distributions.

In [ ]:
for key in scaled_data:
    print(scalings[key])
    for parameter in scaled_data[key][fit_keys]:
        plt.hist(scaled_data[key][parameter], bins=100, density=True, histtype="step", label=parameter)
        plt.legend(loc="best")
    plt.show()
    plt.close()

### Find best fit

Now we have scaled the injections to the fitting space, we can fit the GMMs to the data.
We use the GMM implementation in `scikit-learn`.

In order to identify a suitable number of components to use, we could use any of the three most widely used metrics implemented in `scikit-learn`:
- the score is the mean natural log likelihood over a set of input samples, $S = \left< \ln \mathcal{L} \right>.$
- the Akaike information criterion, $AIC = 2 k - 2 \ln \mathcal{L}.$
- the Bayesian information criterion, $BIC = k \ln N - 2 \ln \mathcal{L}.$

The first is maximized for the "best" model is independent of the number of samples (once a sufficient number of samples have been included) and carries no penalty for using a larger model and so may be more likely to overfit the data.

The other two metrics include a fixed correction for the complexity of the model and are minimized for the "best" model.
The parameter $k$ is the number of parameters of the model and N is the number of input samples.

We use the first metric, however, we split the data into a testing and training set so that we can identify any overfitting.

This cell generates Figure 2 of the paper.

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(6, 4))

for ii, scale_method in enumerate(["empirical"]):
    data = scaled_data[scale_method]
    original_samples = norm.ppf(np.array(data[fit_keys[:]].values))

    n_train = int(len(original_samples) * 0.8)
    n_test = len(original_samples) - n_train
    training = np.random.choice(len(original_samples), n_train, replace=False)
    train = np.zeros(len(original_samples), dtype=bool)
    train[training] = True
    test = ~train

    train_samples = original_samples[train]
    test_samples = original_samples[test]
    train_scores = list()
    test_scores = list()
    all_components = range(1, 21, 1)

    for n_components in tqdm(all_components):
        gmm = GaussianMixture(n_components=n_components).fit(train_samples)
        train_scores.append(gmm.score(train_samples))
        test_scores.append(gmm.score(test_samples))

    plt.sca(axes)
    plt.scatter(all_components, train_scores, label="Train", marker="+", s=40, color=f"C0")
    plt.scatter(all_components, test_scores, label="Test", marker="x", s=40, color=f"C1")
    plt.xlim(1, n_components)
    plt.xlabel("N components")
    plt.ylabel("$\\left<\\ln \\mathcal{D}\\right>$")
    plt.legend(loc="upper left")

plt.tight_layout()
plt.savefig("scaling-vs-components.pdf")
plt.show()
plt.close()

The following are helper functions that will perform the bulk of the analysis.

Many of these are methods to call Gaussian mixture models that is optimized to our purposes (GPU compatibility, caching repeated steps, etc) and closely adapted from `sklearn` code.

In [ ]:
def build_gmms(n_components, kinds=None):
    if kinds is None:
        kinds = list(scaled_data.keys())
    gmms = dict()

    for scale_method in kinds:
        original_samples = norm.ppf(np.array(scaled_data[scale_method][fit_keys[:]].values))
        gmm = GaussianMixture(n_components=n_components).fit(original_samples)
        new_gmm = deepcopy(gmm)
        gmms[scale_method] = CustomGMM(gmm)
    return gmms, test


def draw_samples(scale_method, n_samples=None):
    if n_samples is None:
        n_samples = len(original_samples)
    new_gmm = gmms[scale_method]
    new_samples = unit_normal_cdf(new_gmm.sample(len(original_samples))[0])

    new_data = {
        key: new_samples[:, ii] for ii, key in enumerate(fit_keys)
    }
    new_data = unscale_data(
        new_data.copy(),
        parameters=injection_parameters,
        scale_method=scale_method
    )
    return new_data

In [ ]:
def truncnorm_sample(n_samples, mu, sigma, minimum, maximum):
    val = xp.random.uniform(0, 1, n_samples)
    normalisation = (
        erf((maximum - mu) / 2 ** 0.5 / sigma)
        - erf((minimum - mu) / 2 ** 0.5 / sigma)
    ) / 2
    return erfinv(
        2 * val * normalisation
        + erf((minimum - mu) / 2 ** 0.5 / sigma)
    ) * 2 ** 0.5 * sigma + mu


def unit_normal_cdf(xx):
    return (1 + erf(xx / 2 ** 0.5)) / 2


def unit_normal_ppf(xx):
    return 2 ** 0.5 * erfinv(2 * xx - 1)


def logsumexp(a, axis=None, b=None, keepdims=False, return_sign=False):
    """
    Copied almost verbatim from
    https://github.com/scipy/scipy/blob/v1.5.2/scipy/special/_logsumexp.py#L7-L127
    """
    a_max = xp.amax(a, axis=axis, keepdims=True)

    if a_max.ndim > 0:
        a_max[~xp.isfinite(a_max)] = 0
    elif not xp.isfinite(a_max):
        a_max = 0

    if b is not None:
        b = xp.asarray(b)
        tmp = b * xp.exp(a - a_max)
    else:
        tmp = xp.exp(a - a_max)

    s = xp.sum(tmp, axis=axis, keepdims=keepdims)
    if return_sign:
        sgn = xp.sign(s)
        s *= sgn  # /= makes more sense but we need zero -> zero
    out = xp.log(s)

    if not keepdims:
        a_max = xp.squeeze(a_max, axis=axis)
    out += a_max

    if return_sign:
        return out, sgn
    else:
        return out



def _estimate_gmm_log_gaussian_prob(X, precisions_chol_, means_inner_precisions):
    """Estimate the log Gaussian probability.
    Parameters
    ----------
    X : array-like, shape (n_samples, n_features)
    means : array-like, shape (n_components, n_features)
    precisions_chol : array-like
        Cholesky decompositions of the precision matrices.
        'full' : shape of (n_components, n_features, n_features)
        'tied' : shape of (n_features, n_features)
        'diag' : shape of (n_components, n_features)
        'spherical' : shape of (n_components,)
    covariance_type : {'full', 'tied', 'diag', 'spherical'}
    Returns
    -------
    log_prob : array, shape (n_samples, n_components)
    """
    if X.ndim == 1:
        X = X.reshape(1, -1)
    _, n_features = X.shape

    log_prob = xp.sum(xp.square(
        xp.tensordot(X, precisions_chol_, axes=([1], [1]))
        - means_inner_precisions
    ), axis=-1)

    return -.5 * (n_features * np.log(2 * np.pi) + log_prob)


def _compute_log_det_cholesky(matrix_chol):
    """Compute the log-det of the cholesky decomposition of matrices.
    Parameters
    ----------
    matrix_chol : array-like
        Cholesky decompositions of the matrices.
        'full' : shape of (n_components, n_features, n_features)
        'tied' : shape of (n_features, n_features)
        'diag' : shape of (n_components, n_features)
        'spherical' : shape of (n_components,)
    covariance_type : {'full', 'tied', 'diag', 'spherical'}
    Returns
    -------
    log_det_precision_chol : array-like, shape (n_components,)
        The determinant of the precision matrix for each component.
    """
    n_components, n_features, _ = matrix_chol.shape
    log_det_chol = xp.sum(xp.log(
        matrix_chol.reshape(n_components, -1)[:, ::n_features + 1]
    ), 1)

    return log_det_chol


class CustomGMM(object):
    def __init__(self, gmm, normalization=1):
        self.gmm = gmm
        self.precisions_cholesky_ = xp.asarray(self.gmm.precisions_cholesky_)
        self._logdet = _compute_log_det_cholesky(self.precisions_cholesky_)
        self.means_ = xp.asarray(self.gmm.means_)
        self.ln_weights_ = xp.log(xp.asarray(self.gmm.weights_))
        self.ln_factor = xp.log(1 / normalization)
        self._means_dot_precisions_ = xp.einsum(
            "ik,ikj->ij", self.means_, self.precisions_cholesky_
        )

    def sample(self, n_samples=1):
        samples, components = self.gmm.sample(n_samples=n_samples)
        return xp.asarray(samples), xp.asarray(components)

    def score_samples(self, samples):
        return self.ln_factor + logsumexp(
            _estimate_gmm_log_gaussian_prob(
                samples, self.precisions_cholesky_, self._means_dot_precisions_
            ) + self._logdet + self.ln_weights_,
            axis=1
        )


def unit_normal_ln_pdf(xx):
    return - xx ** 2 / 2 - np.log(2 * np.pi) / 2


def ln_pdf_naive(data, parameters, gmm):
    scaled_data = scale_data(
        data.copy(),
        parameters=parameters,
        scale_method="naive"
    )
    scaled_data = xp.asarray([
        unit_normal_ppf(scaled_data[key])
        for key in scaled_data
    ]).T
    ln_jacobian = (
        - xp.log(parameters["maximum_mass"] - parameters["minimum_mass"])
        - xp.log(1 - parameters["minimum_mass"] / data["mass_1"])
        - powerlaw_ln_pdf(
            data["mass_1"],
            parameters["alpha"],
            parameters["minimum_mass"],
            parameters["maximum_mass"]
        ) - powerlaw_ln_pdf(
            data["mass_ratio"],
            parameters["beta"],
            parameters["minimum_mass"] / data["mass_1"],
            1
        )
    )
    return (
        gmm.score_samples(scaled_data)
        - xp.sum(unit_normal_ln_pdf(scaled_data), axis=-1)
        + ln_jacobian
    )


def ln_pdf_cdf(data, parameters, gmm):
    scaled_data = scale_data(
        data.copy(),
        parameters=parameters,
        scale_method="cdf"
    )
    scaled_data = xp.asarray([
        unit_normal_ppf(scaled_data[key])
        for key in scaled_data
    ]).T
    return (
        gmm.score_samples(scaled_data)
        - xp.sum(unit_normal_ln_pdf(scaled_data), axis=-1)
    )


def ln_pdf_approximate(data, parameters, gmm):
    scaled_data = scale_data(
        data.copy(),
        parameters=parameters,
        scale_method="approximate"
    )
    scaled_data = xp.asarray([
        unit_normal_ppf(scaled_data[key])
        for key in scaled_data
    ]).T
    return (
        gmms["approximate"].score_samples(scaled_data)
        - xp.sum(unit_normal_ln_pdf(scaled_data), axis=-1)
        + 2.35 * xp.log(data["mass_1"])
        + powerlaw_ln_normalization(
            parameters["alpha"] + 2.35, parameters["minimum_mass"], parameters["maximum_mass"]
        )
        - powerlaw_ln_normalization(
            parameters["alpha"], parameters["minimum_mass"], parameters["maximum_mass"]
        )
        + 2 * np.log(data["mass_ratio"])
        + powerlaw_ln_normalization(
            parameters["beta"] + 2, parameters["minimum_mass"] / data["mass_1"], 1
        )
        - powerlaw_ln_normalization(
            parameters["beta"], parameters["minimum_mass"] / data["mass_1"], 1
        )
    )



def ln_pdf_empirical(data, parameters, gmm):
    scaled_data = scale_data(
        data.copy(),
        parameters=parameters,
        scale_method="empirical"
    )
    ln_jacobian = (
        call_interp(data["mass_1"], lnpdf_["mass_1"])
        + call_interp(data["mass_ratio"], lnpdf_["mass_ratio"])
        + call_interp(data["a_1"], lnpdf_["a_1"])
        + call_interp(data["a_2"], lnpdf_["a_2"])
        + call_interp(data["cos_tilt_1"], lnpdf_["cos_tilt_1"])
        + call_interp(data["cos_tilt_2"], lnpdf_["cos_tilt_2"])
        - powerlaw_ln_pdf(
            data["mass_1"],
            parameters["alpha"],
            parameters["minimum_mass"],
            parameters["maximum_mass"]
        ) - powerlaw_ln_pdf(
            data["mass_ratio"],
            parameters["beta"],
            parameters["minimum_mass"] / data["mass_1"],
            1
        )
        + np.log(4)
    )
    scaled_data = xp.asarray([
        unit_normal_ppf(scaled_data[key])
        for key in scaled_data
    ]).T
    return (
        gmms["empirical"].score_samples(scaled_data)
        - xp.sum(unit_normal_ln_pdf(scaled_data), axis=-1)
        + ln_jacobian
    )
        

def ln_pdet(data, parameters, gmms, scale_method):
    funcs = dict(
        naive=ln_pdf_naive, cdf=ln_pdf_cdf, empirical=ln_pdf_empirical, approximate=ln_pdf_approximate
    )
    if scale_method not in funcs:
        raise KeyError(
            f"Scaling method {scale_method} not recognized, should be in {', '.join(funcs.keys())}."
        )
    return funcs[scale_method](data, parameters, gmms[scale_method]) + np.log(fraction_not_hopeless)


## Evaluate the population likelihood

We're finally ready to start evaluating the some population-averaged selection functions.

This uses a mixture of existing code and the functions defined above to do all of the calculations. For details of how these work, see Section 2 of the paper.

First we set up the old style method where we directly resampling the found injections.

In [ ]:
model = Model([two_component_primary_mass_ratio, iid_spin])

_data = load_injection_data("endo3_bbhpop-LIGO-T2100113-v12.hdf5")
with h5py.File("endo3_bbhpop-LIGO-T2100113-v12.hdf5", "r") as ff:
    data = ff["injections"]
    found = np.zeros_like(data["mass1_source"][()], dtype=bool)
    for key in data:
        if "ifar" in key.lower():
            found = found | (data[key][()] > 1)
    _data["prior"] /= xp.asarray(data["redshift_sampling_pdf"][found])
old_vt = ResamplingVT(model=model, data=_data, n_events=69)

13:14 bilby INFO    : Loading VT data from endo3_bbhpop-LIGO-T2100113-v12.hdf5.


Now we generate new samples from a trained GMM to set up the new resampling method.

In [ ]:
gmms, _ = build_gmms(n_components=n_components, kinds=["empirical"])
new_samples_ = draw_samples(scale_method="empirical")
for key in ["total_generated", "analysis_time"]:
    new_samples_[key] = _data[key]
new_samples_["prior"] = xp.exp(
    powerlaw_ln_pdf(
        new_samples_["mass_1"],
        injection_parameters["alpha"],
        injection_parameters["minimum_mass"],
        injection_parameters["maximum_mass"]
    ) + powerlaw_ln_pdf(
        new_samples_["mass_ratio"],
        injection_parameters["beta"],
        injection_parameters["minimum_mass"] / new_samples_["mass_1"],
        1
    )
    - np.log(4)
)

model = Model([two_component_primary_mass_ratio, iid_spin])
new_resampling_vt = ResamplingVT(model=model, data=new_samples_, n_events=69)

Define the prior distribution we will use for the analysis here and the class that will evaluate the population-averaged sensitivity.

In [ ]:
priors = bilby.core.prior.PriorDict(conversion_function=prior_conversion)
priors["alpha"] = bilby.core.prior.Uniform(-2, 8, latex_label="$\\alpha$")
priors["beta"] = bilby.core.prior.Uniform(-2, 8, latex_label="$\\beta$")
priors["mmin"] = bilby.core.prior.Uniform(2, 10, latex_label="$m_{\\min}$")
priors["mmax"] = bilby.core.prior.Uniform(50, 100, latex_label="$m_{\\max}$")
priors["lam"] = bilby.core.prior.Uniform(0, 1, latex_label="$\\lambda_{m}$")
priors["mpp"] = bilby.core.prior.Uniform(20, 50, latex_label="$\\mu_{m}$")
priors["sigpp"] = bilby.core.prior.Uniform(0, 10, latex_label="$\\sigma_{m}$")
priors["amax"] = 1
priors["mu_chi"] = bilby.core.prior.Uniform(0, 1, latex_label="$\\mu_{\\chi}$")
priors["sigma_chi"] = bilby.core.prior.Uniform(0, 0.25, latex_label="$\\sigma^{2}_{\\chi}$")
priors["alpha_chi"] = bilby.core.prior.Constraint(0, 1e6, latex_label="$\\alpha_{\\chi}$")
priors["beta_chi"] = bilby.core.prior.Constraint(0, 1e6, latex_label="$\\beta_{\\chi}$")
priors["xi_spin"] = bilby.core.prior.Uniform(0, 1, latex_label="$\\xi$")
priors["sigma_spin"] = bilby.core.prior.Uniform(0, 4, latex_label="$\\sigma_{s}$")
prior_samples = pd.DataFrame(priors.sample(5000))

prior_samples = prior_conversion(prior_samples)


class GMMVT:
    def __init__(self, n_samples, n_events, method, gmms, sampling_function, injection_parameters):
        self.n_samples = n_samples
        self.n_events = n_events
        self.method = method
        self.gmms = gmms
        self.injection_parameters = injection_parameters
        self.sampling_function = sampling_function

    def __call__(self, parameters):
        mu, n_effective = self.detection_efficiency(parameters)
        if n_effective < 4 * self.n_events:
            return np.inf
        vt_factor = mu / np.exp((3 + self.n_events) / 2 / n_effective)
        return vt_factor
    
    def detection_efficiency(self, parameters):
        parameters["alpha"] *= -1
        if "mmin" not in parameters:
            parameters["mmin"] = parameters.pop("minimum_mass")
        if "mmax" not in parameters:
            parameters["mmax"] = parameters.pop("maximum_mass")
        
        data = self.sampling_function(parameters, self.n_samples)
        ln_pdets = ln_pdet(data, self.injection_parameters, self.gmms, self.method)
        weights = xp.exp(ln_pdets)
        mu = float(xp.mean(weights))
        neff = xp.sum(weights) ** 2 / xp.sum(weights ** 2)
        return mu, neff


def sample_model(parameters, n_samples):
    data = dict()
    n_peak = int(n_samples * parameters["lam"])
    n_powerlaw = n_samples - n_peak
    data["mass_1"] = xp.random.permutation(np.concatenate([
        powerlaw_sample(parameters["alpha"], parameters["mmin"], parameters["mmax"], n_powerlaw),
        truncnorm_sample(n_samples=n_peak, minimum=parameters["mmin"], maximum=100, mu=parameters["mpp"], sigma=parameters["sigpp"])
    ]))
    data["mass_ratio"] = powerlaw_sample(parameters["beta"], parameters["mmin"] / data["mass_1"], 1, n_samples)
    for key in ["a_1", "a_2"]:
        data[key] = (xp.random.beta(parameters["alpha_chi"], parameters["beta_chi"], n_samples) * 0.99998 + 0.00001) * parameters.get("amax", 1)
    n_normal = int(n_samples * parameters["xi_spin"])
    n_isotropic = n_samples - n_normal
    for key in ["cos_tilt_1", "cos_tilt_2"]:
        data[key] = xp.concatenate([
            xp.random.uniform(-1, 1, n_isotropic),
            truncnorm_sample(n_samples=n_normal, mu=1, sigma=parameters["sigma_spin"], minimum=-1, maximum=1)
        ])
    return data

Evaluate $P_{\rm det}$ using the two injection resampling methods.

In [ ]:
pdets = list()
n_effs_old = list()
pdets_rw = list()
n_effs_old_rw = list()
for ii in tqdm(range(len(prior_samples[:]))):
    parameters = dict(prior_samples.iloc[ii])
    if "mmin" not in parameters:
        parameters["mmin"] = parameters.pop("minimum_mass")
    if "mmax" not in parameters:
        parameters["mmax"] = parameters.pop("maximum_mass")
    mu, var = old_vt.detection_efficiency(parameters=parameters)
    if mu > 0:
        pdets.append(mu)
        n_effs_old.append(mu ** 2 / var)
    else:
        pdets.append(np.nan)
        n_effs_old.append(np.nan)
    mu, var = new_resampling_vt.detection_efficiency(parameters=parameters)
    if mu > 0:
        pdets_rw.append(mu)
        n_effs_old_rw.append(mu ** 2 / var)
    else:
        pdets_rw.append(np.nan)
        n_effs_old_rw.append(np.nan)

Evaluate $P_{\rm det}$ for our new method.

In [ ]:
n_samples = 10000
method = "empirical"

ln_pdets = dict()
n_effs = dict()
_ln_pdets = list()
_n_effs = list()
gmm_vt = GMMVT(
    n_samples,
    n_events=69,
    method="empirical",
    gmms=gmms,
    sampling_function=sample_model,
    injection_parameters=injection_parameters
)
for ii in tqdm(range(len(prior_samples[:]))):
    parameters = dict(prior_samples.iloc[ii])
    if "mmin" not in parameters:
        parameters["mmin"] = parameters.pop("minimum_mass")
    if "mmax" not in parameters:
        parameters["mmax"] = parameters.pop("maximum_mass")
    mu, n_eff = gmm_vt.detection_efficiency(parameters=parameters)
    _ln_pdets.append(np.log(mu))
    _n_effs.append(n_eff)
_ln_pdets = np.array(_ln_pdets)
_n_effs = np.array(_n_effs)
ln_pdets[n_components] = _ln_pdets
n_effs[n_components] = _n_effs

Plot the difference between the $P_{\rm det}$ computed with the new methods and the two injection resampling method.

The similarity in the trends between the original samples and the samples drawn from the GMM fit demsontrates that the differences are due to the difference in evaluation method, rather than loss of information in the GMM fitting.

In [ ]:
for param in list(priors.keys()):
    plt.figure(figsize=(12, 7.5))
    plt.scatter(
        prior_samples[param],
        np.log10(np.exp(_ln_pdets - np.log(pdets))),
        label="Old",
        s=20,
        marker="x"
    )
    plt.scatter(
        prior_samples[param],
        np.log10(np.exp(_ln_pdets - np.log(pdets_rw))),
        label="New Samples",
        s=20,
        marker="+"
    )
    plt.xlabel(priors[param].latex_label)
    plt.ylabel("$\\Delta \\log_{10} P_{\\rm det}$")
    if param in ["alpha_chi", "beta_chi"]:
        plt.xscale("log")
    plt.show()
    plt.close()

Plot the distribution of selection functions for the three methods for the non-singular and singular spin distributions.

This is Figure 3 of the paper.

In [ ]:
non_singular = (prior_samples["alpha_chi"].values > 1) & (prior_samples["beta_chi"].values > 1)

fig, axes = plt.subplots(nrows=2, figsize=(8, 10))

plt.sca(axes[0])
plt.hist(np.log10(pdets)[non_singular], bins=np.linspace(-4, -1, 100), density=True, histtype="step")
plt.hist(np.log10(pdets_rw)[non_singular], bins=np.linspace(-4, -1, 100), density=True, histtype="step")
plt.hist(np.log10(np.exp(_ln_pdets[non_singular])), bins=np.linspace(-4, -1, 100), density=True, histtype="step")
plt.yscale("log")
plt.xlabel("$\\log_{10} P_{\\rm det}$")
plt.ylabel("$p(\\log_{10} P_{\\rm det})$")

plt.sca(axes[1])
plt.hist(np.log10(pdets)[~non_singular], bins=np.linspace(-9, -1, 100), density=True, histtype="step", label="Old")
plt.hist(np.log10(pdets_rw)[~non_singular], bins=np.linspace(-9, -1, 100), density=True, histtype="step", label="New Samples")
plt.hist(np.log10(np.exp(_ln_pdets[~non_singular])), bins=np.linspace(-9, -1, 100), density=True, histtype="step", label="New")
plt.legend(loc="upper left")
plt.yscale("log")
plt.xlabel("$\\log_{10} P_{\\rm det}$")
plt.ylabel("$p(\\log_{10} P_{\\rm det})$")
plt.tight_layout()
plt.savefig("Pdet.pdf")
plt.show()
plt.close()

Plot the original samples weighted by the selection function for the different methods.

Using the arbitrary scale more clearly shows the difference for the spin magnitude parameters. Setting `density=True` demonstrates that there is no observable difference for many of the parameters.

One of these is Figure 4 in the paper.

In [ ]:
for parameter in priors.keys():
    plt.figure(figsize=(8, 5))
    if parameter in ["alpha_chi", "beta_chi"]:
        points = np.log10(prior_samples[parameter])
    else:
        points = prior_samples[parameter]
    for ii, samples_ in enumerate([pdets, pdets_rw, np.exp(_ln_pdets)]):
        plt.hist(
            points,
            weights=samples_,
            density=False,
            histtype="step",
            color=f"C{ii}",
            bins=100,
            label=["Old", "New Samples", "New"][ii]
        )
    if parameter in ["alpha_chi", "beta_chi"]:
        plt.xlabel(f"$\\log_{{10}}\,${priors[parameter].latex_label}")
    else:
        plt.xlabel(priors[parameter].latex_label)
    plt.ylabel("$P_{\\rm det}$ [arbitrary units]")
    plt.legend(loc="best")
    plt.tight_layout()
    plt.savefig(f"Pdet_{parameter}.pdf")
    # plt.yscale("log")
    # plt.ylim(-5, 1)
    plt.show()
    plt.close()

Plot the number of effective samples as a function of population parameter for the three methods.

Two of these are used in Figure 5 of the paper.

In [ ]:
method = "empirical"
for param in list(priors.keys()):
    plt.figure(figsize=(12, 7.5))
    for ii, _neffs in enumerate([n_effs_old, n_effs_old_rw, _n_effs]):
        plt.scatter(
            prior_samples[param],
            _neffs,
            s=20,
            label=["Old", "New Samples", "New"][ii],
            color=f"C{ii}",
        )
    plt.ylim(1, 10 * max(n_effs_old))
    plt.axhline(4 * 69, color="k", linestyle="--", label="Threshold")
    if param in priors:
        plt.xlabel(priors[param].latex_label)
    else:
        plt.xlabel(param.replace("_", " "))
    if param in ["alpha_chi", "beta_chi"]:
        plt.xscale("log")
    plt.ylabel("$N_{\\rm eff}$")
    plt.legend(loc="upper left", ncol=2)
    plt.yscale("log")
    plt.tight_layout()
    plt.savefig(f"neff_{param}.pdf")
    plt.show()
    plt.close()

In [ ]:
for meth, neff in zip(["Old", "New Samples", "New"], [n_effs_old, n_effs_old_rw, _neffs]):
    print(
        f"For the {meth} method we reject "
        f"{np.mean(np.array(neff)[non_singular] < 4 * 69) * 100:.1f}% of non-singular samples, "
        f"{np.mean(np.array(neff)[~non_singular] < 4 * 69) * 100:.1f}% of singular samples, "
        f"and {np.mean(np.array(neff) < 4 * 69) * 100:.1f}% of all samples."
    )


print(f"New method has more effective samples (smaller uncertainty) in {np.mean(np.array(_neffs) > np.array(n_effs_old)) * 100:.1f}% of the space.")

For the Old method we reject 12.6% of non-singular samples, 53.1% of singular samples, and 44.2% of all samples.
For the New Samples method we reject 12.9% of non-singular samples, 60.5% of singular samples, and 49.9% of all samples.
For the New method we reject 0.1% of non-singular samples, 0.4% of singular samples, and 0.3% of all samples.
New method has more effective samples (smaller uncertainty) in 82.6% of the space.


## Run population inference

Now we can run population inference with our three selection function evaluators.

Here I mount my drive and use a prepared file. Other users will have to collate the data from the official releases.

For this demonstration I use the `bilby-mcmc` sampler with fairly aggresive settings as all we are interested in are the observed spectra and don't mind having correlated MCMC samples.

In [ ]:
from gwpopulation_pipe.utils import MinimumEffectiveSamplesLikelihood
from gwpopulation.conversions import convert_to_beta_parameters
from gwpopulation.hyperpe import HyperparameterLikelihood

likelihood_class = MinimumEffectiveSamplesLikelihood

posteriors = pd.read_pickle("drive/MyDrive/science/GWTC-3_posteriors.pkl")

run_model = Model([two_component_primary_mass_ratio, iid_spin])

likelihood_original = likelihood_class(
    posteriors=deepcopy(posteriors),
    hyper_prior=deepcopy(run_model),
    conversion_function=convert_to_beta_parameters,
    selection_function=old_vt
)

likelihood_new_samples = likelihood_class(
    posteriors=deepcopy(posteriors),
    hyper_prior=deepcopy(run_model),
    conversion_function=convert_to_beta_parameters,
    selection_function=new_resampling_vt
)

likelihood_new = likelihood_class(
    posteriors=deepcopy(posteriors),
    hyper_prior=deepcopy(run_model),
    conversion_function=convert_to_beta_parameters,
    selection_function=gmm_vt
)

In [ ]:
result_new = bilby.run_sampler(
    likelihood=likelihood_new, priors=priors,
    nsamples=500, proposal_cycle="defaultnoKDnoNFnoUN",
    L1Steps=10, L2Steps=1, printdt=20, check_point_delta_t=180,
    fixed_tau=2, fixed_discard=100,
    nlive=250, label="new", resume=False, sampler="bilby_mcmc"
)
result_original = bilby.run_sampler(
    likelihood=likelihood_original, priors=priors,
    nsamples=500, proposal_cycle="defaultnoKDnoNFnoUN",
    L1Steps=10, L2Steps=1, printdt=20, check_point_delta_t=180,
    fixed_tau=2, fixed_discard=100,
    nlive=250, label="original", resume=False, sampler="bilby_mcmc"
)
result_new_samples = bilby.run_sampler(
    likelihood=likelihood_new_samples, priors=priors,
    nsamples=500, proposal_cycle="defaultnoKDnoNFnoUN",
    L1Steps=10, L2Steps=1, printdt=20, check_point_delta_t=180,
    fixed_tau=2, fixed_discard=100,
    nlive=250, label="new_samples", resume=False, sampler="bilby_mcmc"
)

Plot the inferred primary mass and spin magnitude spectra.

This is Figure 6 in the paper.

In [ ]:
aa = np.linspace(1e-5, 1 - 1e-5, 10000)
ms = np.linspace(2, 100, 10000)


def _truncnorm_pdf(xx, mu, sigma, high, low):
    from scipy.special import erf, erfinv
    norm = 2 ** 0.5 / xp.pi ** 0.5 / sigma
    norm /= erf((high - mu) / 2 ** 0.5 / sigma) + erf((mu - low) / 2 ** 0.5 / sigma)
    prob = np.exp(-np.power(xx - mu, 2) / (2 * sigma ** 2))
    prob *= norm
    prob *= (xx <= high) & (xx >= low)
    return prob


plt.figure(figsize=(8, 5))
fig, axes = plt.subplots(nrows=2, figsize=(8, 10))
for jj, result in enumerate([result_original, result_new_samples, result_new]):
    a_lines = list()
    m_lines = list()
    for ii in trange(len(result.posterior)):
        parameters = dict(result.posterior.iloc[ii])
        parameters = convert_to_beta_parameters(parameters)[0]
        prob = np.exp(
            (parameters["alpha_chi"] - 1) * np.log(aa)
            + (parameters["beta_chi"] - 1) * np.log(1 - aa)
            - scbetaln(parameters["alpha_chi"], parameters["beta_chi"])
        )
        a_lines.append(prob)
        prob = (
            (1 - parameters["lam"]) * np.exp(powerlaw_ln_pdf(ms, -parameters["alpha"], parameters["mmin"], parameters["mmax"]))
            + parameters["lam"] * _truncnorm_pdf(ms, parameters["mpp"], parameters["sigpp"], 100, parameters["mmin"])
        )
        m_lines.append(prob)
    plt.sca(axes[0])
    plt.plot(ms, np.mean(m_lines, axis=0), color=f"C{jj}", label=result.label.replace("_", " ").title())
    plt.fill_between(ms, np.percentile(m_lines, 5, axis=0), np.percentile(m_lines, 95, axis=0), color=f"C{jj}", alpha=0.1)
    plt.sca(axes[1])
    plt.plot(aa, np.mean(a_lines, axis=0), color=f"C{jj}", label=result.label.replace("_", " ").title())
    plt.fill_between(aa, np.percentile(a_lines, 5, axis=0), np.percentile(a_lines, 95, axis=0), color=f"C{jj}", alpha=0.1)
plt.sca(axes[0])
plt.yscale("log")
plt.xlim(2, 100)
plt.legend(loc="upper right")
plt.xlabel("$m_{1}\,[M_{\\odot}]$")
plt.ylabel("$p(m_{1})$")
plt.sca(axes[1])
plt.xlim(0, 1)
plt.ylim(0, 5)
plt.xlabel("$a$")
plt.ylabel("$p(a)$")
plt.tight_layout()
plt.savefig("ppd-m1-a.pdf")
plt.show()
plt.close()